# PySpark: An Overview

## What is PySpark?

PySpark is the **Python API for Apache Spark**, an open-source, distributed computing framework designed for processing large-scale datasets across multiple machines. It combines the simplicity of Python with Spark’s ability to handle **big data processing, machine learning, and streaming workloads**.

Instead of running computations on a single machine (like Pandas or NumPy), PySpark distributes data and computations across a **cluster of machines**, enabling faster execution on large datasets.

---

## Who Uses PySpark?

1. **Data Engineers**

   * To process, clean, and transform massive datasets (structured & unstructured).
   * To build data pipelines for ETL (Extract, Transform, Load).

2. **Data Scientists & Machine Learning Engineers**

   * For feature engineering and model training on large datasets.
   * Using **MLlib**, Spark’s machine learning library.

3. **Analysts & Business Intelligence Teams**

   * To query large datasets that don’t fit in memory.
   * To run SQL-like queries via **Spark SQL**.

4. **Companies & Industries**

   * **Tech Giants**: Netflix, Uber, Airbnb use PySpark for recommendation systems, real-time analytics.
   * **Finance**: Fraud detection, risk analysis.
   * **Retail/E-commerce**: Customer analytics, personalization.
   * **Healthcare & Pharma**: Genomics, patient data processing.

---

## When to Use PySpark?

* **Big Data Scenarios**: When datasets are too large to fit in a single machine’s memory (e.g., terabytes of logs).
* **Distributed Computing**: When parallelism is required across multiple nodes.
* **Real-Time Data Processing**: Using **Spark Streaming** for handling continuous data (e.g., IoT sensors, financial transactions).
* **Integration with Big Data Ecosystem**: When working with Hadoop (HDFS), Hive, or cloud storage like AWS S3, Azure Data Lake.
* **Scalability Needs**: When the workload might grow from gigabytes to terabytes.

You **don’t need PySpark** if your dataset fits comfortably in memory on a single machine and can be handled with Pandas or NumPy.

---

## PySpark vs Pandas

| Feature             | **Pandas**                                                          | **PySpark**                                                                     |
| ------------------- | ------------------------------------------------------------------- | ------------------------------------------------------------------------------- |
| **Scale**           | Works on a single machine, limited by RAM.                          | Distributed computing across many machines, scales to terabytes.                |
| **Performance**     | Fast for small to medium data (MBs–GBs).                            | Optimized for big data (GBs–PBs).                                               |
| **Execution Model** | In-memory operations, eager execution.                              | Lazy execution (builds DAG of tasks, executes when triggered).                  |
| **Data Size Limit** | Typically fits in system RAM (few GBs).                             | Handles much larger data (hundreds of GBs to TBs).                              |
| **Use Cases**       | Data exploration, small-scale analysis, prototyping.                | Production-level big data pipelines, machine learning at scale, streaming data. |
| **Learning Curve**  | Easier, intuitive API.                                              | Steeper, requires understanding distributed systems.                            |
| **Integration**     | Works well with Python libraries (NumPy, Scikit-learn, Matplotlib). | Integrates with Hadoop ecosystem, Spark MLlib, Spark SQL.                       |

---

## Key Takeaways

* Use **Pandas** when your dataset fits in memory and you need **fast prototyping, visualization, or small-scale analysis**.
* Use **PySpark** when dealing with **large-scale, distributed, or real-time data processing**.
* Both tools can complement each other:

  * Often data is **sampled and explored with Pandas**, then scaled to production with PySpark.

---


# PySpark Introduction: Hands-on Notebook

This notebook covers three parts with **15 examples each**:

1. **Spark architecture and components**  
2. **RDDs vs DataFrames**  
3. **Setting up PySpark**  

### How to use this notebook
- If you're on **local machine**, ensure Java (8/11/17), Python 3.x, and PySpark are installed.
- If you're on **Colab**, first run the **Setup cell** in Part 3 (it installs Java and PySpark and creates the `SparkSession`).

For every example:
- We start with a brief **function/concept description** (markdown).
- Then we provide **code** you can run.
- Finally, we include **explanations and optimization tips** in a markdown block following the code.


# Part 1 — Spark architecture and components (15 examples)

## 1) SparkSession: entry point

**What it is:** `SparkSession` is the unified entry point to Spark functionality in PySpark. It encapsulates `SparkContext`, `SQLContext`, and Hive support (optionally).

In [1]:
# Create a SparkSession (do this once per notebook)
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName("PySpark-Intro-Architecture-1")
         .master("local[*]")
         .getOrCreate())

print("Spark version:", spark.version)
print("App name:", spark.sparkContext.appName)
print("Master:", spark.sparkContext.master)

Spark version: 3.5.1
App name: PySpark-Intro-Architecture-1
Master: local[*]


**Explanation:**  
- `SparkSession.builder` configures the app (name, master URL, configs).  
- `master("local[*]")` uses all local cores.  
- `getOrCreate()` returns existing session if present.

**Optimize:**  
- Reuse a single `SparkSession` in your notebook/job.  
- Set configs **before** creating `SparkSession` (e.g., shuffle partitions, serializer).

## 2) SparkContext and low-level access

**What it is:** `SparkContext` is the core connection to the cluster; it handles executors, tasks, stages.

In [2]:
sc = spark.sparkContext
print("Application ID:", sc.applicationId)
print("Default parallelism:", sc.defaultParallelism)
print("UI Web URL:", sc.uiWebUrl)  # None in some environments

Application ID: local-1755777824582
Default parallelism: 2
UI Web URL: http://8e72ff8dbcad:4040


**Explanation:**  
- `applicationId` identifies your app in Spark UI / History Server.  
- `defaultParallelism` is a good hint for partition counts.  
- `uiWebUrl` gives the Spark UI address (may be `None` in restricted envs).

**Optimize:**  
- Keep partition counts near `defaultParallelism` for CPU-bound tasks.  
- Monitor jobs via Spark UI to detect skew/shuffles.

## 3) SparkConf: configuration

**What it is:** `SparkConf` stores key-value configs that tune Spark runtime (memory, serializers, shuffle partitions, etc.).

In [3]:
from pyspark import SparkConf

conf = spark.sparkContext.getConf()
print("Some confs:")
for k in ["spark.app.name", "spark.master", "spark.sql.shuffle.partitions"]:
    try:
        print(k, "=", conf.get(k))
    except:
        pass

# Setting a conf dynamically (takes effect for new sessions or some runtime tunables)
spark.conf.set("spark.sql.shuffle.partitions", "8")
print("Updated spark.sql.shuffle.partitions:", spark.conf.get("spark.sql.shuffle.partitions"))

Some confs:
spark.app.name = PySpark-Intro-Architecture-1
spark.master = local[*]
spark.sql.shuffle.partitions = None
Updated spark.sql.shuffle.partitions: 8


**Explanation:**  
- Use `SparkConf` to read and set configs. Some require session restart.  
- `spark.sql.shuffle.partitions` controls number of reduce-side partitions after wide operations.

**Optimize:**  
- Reduce `spark.sql.shuffle.partitions` for small data to avoid overhead.  
- Increase for large shuffles to improve parallelism (watch overhead).

## 4) Driver vs Executors

**What it is:** The **Driver** runs your Python code and manages the DAG. **Executors** run tasks on partitions.

In [7]:
# Inspect executor status (in local mode shows only driver/executor)
# Note: Detailed memory status is best monitored via the Spark UI in this environment.
status_tracker = spark.sparkContext.statusTracker
# In local mode, detailed executor memory details might not be fully exposed via the statusTracker.
print("Executor status information might be limited in local mode.")
# For detailed memory monitoring, use the Spark UI.

Executor status information might be limited in local mode.


**Explanation:**  
- Executors are the worker JVMs.  
- `getExecutorMemoryStatus()` helps you see memory distribution.

**Optimize:**  
- Size executors for your workload (cores, memory).  
- Prefer fewer, larger executors for shuffle-heavy jobs; more, smaller for parallel I/O.

## 5) Stages, Tasks, and DAG (via `explain`)

**What it is:** Spark builds a **logical plan** then a **physical plan** across stages/tasks. You can inspect with `explain()`.

In [8]:
from pyspark.sql import functions as F

df = spark.range(0, 100000).withColumn("v", (F.rand()*100).cast("double"))
df2 = df.where(F.col("id") % 2 == 0).groupBy((F.col("v") > 50).alias("gt50")).count()
df2.explain(True)  # extended

== Parsed Logical Plan ==
'Aggregate [('v > 50) AS gt50#5], [('v > 50) AS gt50#5, count(1) AS count#9L]
+- Filter ((id#0L % cast(2 as bigint)) = cast(0 as bigint))
   +- Project [id#0L, cast((rand(-8527907505096746518) * cast(100 as double)) as double) AS v#2]
      +- Range (0, 100000, step=1, splits=Some(2))

== Analyzed Logical Plan ==
gt50: boolean, count: bigint
Aggregate [(v#2 > cast(50 as double))], [(v#2 > cast(50 as double)) AS gt50#5, count(1) AS count#9L]
+- Filter ((id#0L % cast(2 as bigint)) = cast(0 as bigint))
   +- Project [id#0L, cast((rand(-8527907505096746518) * cast(100 as double)) as double) AS v#2]
      +- Range (0, 100000, step=1, splits=Some(2))

== Optimized Logical Plan ==
Aggregate [_groupingexpression#12], [_groupingexpression#12 AS gt50#5, count(1) AS count#9L]
+- Project [(v#2 > 50.0) AS _groupingexpression#12]
   +- Filter ((id#0L % 2) = 0)
      +- Project [id#0L, (rand(-8527907505096746518) * 100.0) AS v#2]
         +- Range (0, 100000, step=1, splits=

**Explanation:**  
- `explain(True)` shows parsed, analyzed, optimized, and physical plans.  
- Shuffles indicate stage boundaries.

**Optimize:**  
- Avoid unnecessary shuffles (e.g., use `reduceByKey`, `mapPartitions`).  
- Repartition early to balance skew; `salting` keys can help.

## 6) Storage: cache, persist, unpersist

**What it is:** Store intermediate results in memory/disk for reuse with `cache()`/`persist()` and clean with `unpersist()`.

In [9]:
from pyspark import StorageLevel

df = spark.range(0, 1_000_00).withColumnRenamed("id","n")
df_cached = df.persist(StorageLevel.MEMORY_ONLY)
print(df_cached.count())  # materialize
print(df_cached.storageLevel)
df_cached.unpersist()

100000
Memory Serialized 1x Replicated


DataFrame[n: bigint]

**Explanation:**  
- `persist(StorageLevel)` lets you choose memory/disk/serialization.  
- First action materializes the cache.

**Optimize:**  
- Cache only when reused many times.  
- Use `MEMORY_AND_DISK` for large data; `OFF_HEAP` if configured.

## 7) Accumulators: global counters

**What it is:** Accumulators aggregate information across tasks (write-only in executors).

In [13]:
# from pyspark.accumulators import LongAccumulator # Removed explicit import

# acc = spark.sparkContext.accumulator(0, LongAccumulator()) # Modified accumulator creation
acc = spark.sparkContext.accumulator(0)

rdd = spark.sparkContext.parallelize(["ok","err","ok","err","ok"], 2)
def f(x):
    if x == "err":
        acc.add(1)
    return x

print(rdd.map(f).collect())
print("Errors seen:", acc.value)

['ok', 'err', 'ok', 'err', 'ok']
Errors seen: 2


**Explanation:**  
- Accumulators are useful for metrics (e.g., bad rows).  
- Writes happen on executors; driver reads final value.

**Optimize:**  
- Use sparingly; side effects complicate retries.  
- Prefer DataFrame metrics when possible.

## 8) Broadcast variables

**What it is:** Broadcast small lookup data to executors efficiently, instead of shipping with each task.

In [14]:
lookup = {"IN":"India","US":"United States","DE":"Germany"}
bc = spark.sparkContext.broadcast(lookup)

rdd = spark.sparkContext.parallelize(["IN","US","XX"])
def map_country(code):
    return bc.value.get(code, "Unknown")

print(rdd.map(map_country).collect())

['India', 'United States', 'Unknown']


**Explanation:**  
- Broadcast once, re-used by all tasks/executors.  
- Ideal for small dimension tables.

**Optimize:**  
- For DataFrames, enable **broadcast joins** with `broadcast(dim_df)`.  
- Keep broadcast size small (< ~10s of MB).

## 9) Partitioning fundamentals

**What it is:** Data is split into partitions processed in parallel. Partition counts affect parallelism and overhead.

In [15]:
rdd = spark.sparkContext.parallelize(range(100), numSlices=4)
print("Partitions:", rdd.getNumPartitions())
rdd2 = rdd.repartition(8)
print("After repartition:", rdd2.getNumPartitions())
rdd3 = rdd2.coalesce(2)
print("After coalesce:", rdd3.getNumPartitions())

Partitions: 4
After repartition: 8
After coalesce: 2


**Explanation:**  
- `repartition(n)` reshuffles to increase/decrease partitions.  
- `coalesce(n)` tries to shrink partitions without a full shuffle.

**Optimize:**  
- Increase partitions for I/O parallelism; decrease to reduce small-files/problem.  
- For wide ops, align partitions with downstream needs.

## 10) Checkpointing

**What it is:** Checkpoint truncates lineage to stable storage to avoid long dependency chains (useful in streaming/iterative).

In [16]:
import tempfile
tmp_dir = tempfile.mkdtemp(prefix="spchkpt_")
spark.sparkContext.setCheckpointDir(tmp_dir)

rdd = spark.sparkContext.range(0, 100000).map(lambda x: (x, x%10)).mapValues(lambda v: v*v)
rdd.checkpoint()  # mark for checkpoint
print(rdd.count())  # materialize checkpoint
print("Checkpoint dir:", tmp_dir)

100000
Checkpoint dir: /tmp/spchkpt_imcok0wn


**Explanation:**  
- Long lineage can cause recomputation on failures; checkpoint cuts it.  
- Needs a **reliable storage** (HDFS/S3) in production.

**Optimize:**  
- Use checkpoint judiciously (I/O heavy).  
- Persist + checkpoint for performance & fault tolerance.

## 11) Serialization and Python <-> JVM

**What it is:** Spark runs executors on JVM; PySpark serializes data between Python and JVM; avoid excessive row-wise UDFs.

In [17]:
from pyspark.sql import functions as F

df = spark.range(0, 1000).withColumn("x", (F.rand()*100).cast("double"))
# Python UDF (slower) vs native expression (faster)
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf

@udf(DoubleType())
def py_square(v):
    return float(v)*float(v) if v is not None else None

df_py = df.withColumn("py_sq", py_square("x"))
df_native = df.withColumn("sq", F.col("x")*F.col("x"))
df_py.explain()
df_native.explain()

== Physical Plan ==
*(2) Project [id#55L, x#57, pythonUDF0#69 AS py_sq#61]
+- BatchEvalPython [py_square(x#57)#60], [pythonUDF0#69]
   +- *(1) Project [id#55L, (rand(5032837550182834869) * 100.0) AS x#57]
      +- *(1) Range (0, 1000, step=1, splits=2)


== Physical Plan ==
*(1) Project [id#55L, x#57, (x#57 * x#57) AS sq#65]
+- *(1) Project [id#55L, (rand(5032837550182834869) * 100.0) AS x#57]
   +- *(1) Range (0, 1000, step=1, splits=2)




**Explanation:**  
- Python UDFs cross the Python/JVM boundary and inhibit optimizations.  
- Native Spark SQL functions are vectorized and optimized.

**Optimize:**  
- Prefer built-in functions or SQL expressions over UDFs.  
- If needed, use **Pandas UDFs** with Arrow for batch speedups.

## 12) Shuffle and skew awareness

**What it is:** Shuffles redistribute data; skewed keys cause stragglers and long stages.

In [18]:
from pyspark.sql import functions as F

data = [(i, 1 if i==0 else i%5) for i in range(100000)]
df = spark.createDataFrame(data, ["id","key"])
agg = df.groupBy("key").count()
agg.explain()  # watch for Exchange nodes

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[key#71L], functions=[count(1)])
   +- Exchange hashpartitioning(key#71L, 8), ENSURE_REQUIREMENTS, [plan_id=129]
      +- HashAggregate(keys=[key#71L], functions=[partial_count(1)])
         +- Project [key#71L]
            +- Scan ExistingRDD[id#70L,key#71L]




**Explanation:**  
- `Exchange` in plan indicates shuffles.  
- Few hot keys → skew → one or few partitions take most work.

**Optimize:**  
- Pre-sample to find skew; use **salting** or **skew join hints**.  
- Enable AQE (Adaptive Query Execution) in Spark 3 for better join/shuffle decisions.

## 13) Catalyst optimizer (SQL/DataFrame)

**What it is:** Catalyst transforms logical plans to optimized physical plans; `explain()` reveals optimizations.

In [19]:
from pyspark.sql import functions as F

df = spark.range(0, 50000).select(F.col("id").alias("n"))
df_opt = df.where(F.col("n")>100).where(F.col("n")<2000).select(F.col("n")+1)
df_opt.explain(True)  # predicate pushdown, constant folding, projection pruning

== Parsed Logical Plan ==
'Project [unresolvedalias(('n + 1), Some(org.apache.spark.sql.Column$$Lambda$3489/0x0000000841460840@1d788876))]
+- Filter (n#84L < cast(2000 as bigint))
   +- Filter (n#84L > cast(100 as bigint))
      +- Project [id#82L AS n#84L]
         +- Range (0, 50000, step=1, splits=Some(2))

== Analyzed Logical Plan ==
(n + 1): bigint
Project [(n#84L + cast(1 as bigint)) AS (n + 1)#86L]
+- Filter (n#84L < cast(2000 as bigint))
   +- Filter (n#84L > cast(100 as bigint))
      +- Project [id#82L AS n#84L]
         +- Range (0, 50000, step=1, splits=Some(2))

== Optimized Logical Plan ==
Project [(id#82L + 1) AS (n + 1)#86L]
+- Filter ((id#82L > 100) AND (id#82L < 2000))
   +- Range (0, 50000, step=1, splits=Some(2))

== Physical Plan ==
*(1) Project [(id#82L + 1) AS (n + 1)#86L]
+- *(1) Filter ((id#82L > 100) AND (id#82L < 2000))
   +- *(1) Range (0, 50000, step=1, splits=2)



**Explanation:**  
- Chains of filters/projections are merged, constants folded, and columns pruned.  
- Datasource filters can be pushed down (Parquet/CSV/Hive).

**Optimize:**  
- Select only needed columns early.  
- Use columnar formats (Parquet/ORC) for pushdown & pruning.

## 14) Adaptive Query Execution (AQE)

**What it is:** AQE adapts the plan at runtime based on stats (e.g., coalescing shuffles, changing join strategy).

In [20]:
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", "true")

from pyspark.sql import functions as F
df1 = spark.range(0, 1_000_0).withColumn("k", (F.rand()*10).cast("int"))
df2 = spark.range(0, 1_000_0).withColumn("k", (F.rand()*10).cast("int"))
joined = df1.join(df2, "k")
joined.explain()  # with AQE, observe coalesced shuffle partitions

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [k#90, id#88L, id#93L]
   +- BroadcastHashJoin [k#90], [k#95], Inner, BuildRight, false
      :- Filter isnotnull(k#90)
      :  +- Project [id#88L, cast((rand(-3872075538131230837) * 10.0) as int) AS k#90]
      :     +- Range (0, 10000, step=1, splits=2)
      +- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[1, int, false] as bigint)),false), [plan_id=175]
         +- Filter isnotnull(k#95)
            +- Project [id#93L, cast((rand(-4770414836113491682) * 10.0) as int) AS k#95]
               +- Range (0, 10000, step=1, splits=2)




**Explanation:**  
- AQE adjusts partition counts and join types at runtime.  
- Helps with skew and reduces small post-shuffle partitions.

**Optimize:**  
- Enable AQE in Spark 3+.  
- Combine with broadcast hints and stats for best results.

## 15) Monitoring & Logging

**What it is:** Spark UI/History Server expose stages, tasks, storage, environment, and SQL tabs. Logging helps troubleshoot.

In [21]:
# Adjust log level to reduce noise
spark.sparkContext.setLogLevel("WARN")

# Print a helpful link (if available in your environment)
print("Spark UI:", spark.sparkContext.uiWebUrl)

Spark UI: http://8e72ff8dbcad:4040


**Explanation:**  
- Use **Spark UI** to inspect failures, skew, GC, and SQL plans.  
- History Server shows completed apps (enable event logging via conf).

**Optimize:**  
- Set `spark.eventLog.enabled=true` and `spark.eventLog.dir` for production.  
- Use structured logging & metrics (Prometheus, Ganglia) for observability.

# Part 2 — RDDs vs DataFrames (15 examples)

## 1) Creating RDD vs DataFrame

**Goal:** Show basic construction from Python collections.

In [22]:
data = [("alice", 34), ("bob", 42), ("cathy", 29)]
rdd = spark.sparkContext.parallelize(data, 2)
df = spark.createDataFrame(data, ["name","age"])

print("RDD first:", rdd.first())
df.show()

RDD first: ('alice', 34)
+-----+---+
| name|age|
+-----+---+
|alice| 34|
|  bob| 42|
|cathy| 29|
+-----+---+



**Explanation:**  
- RDD is an untyped collection of Python objects.  
- DataFrame has schema (`name`, `age`) enabling Catalyst optimization.

**Optimize:**  
- Prefer DataFrames for SQL ops and Tungsten columnar execution.  
- Use RDDs only for low-level transformations not expressible in SQL.

## 2) map vs select/expr

**Goal:** Compare element-wise transformations.

In [23]:
from pyspark.sql import functions as F

rdd2 = rdd.map(lambda x: (x[0].upper(), x[1]+1))
df2 = df.select(F.upper(F.col("name")).alias("NAME"), (F.col("age")+1).alias("age_plus_one"))

print(rdd2.take(2))
df2.show()

[('ALICE', 35), ('BOB', 43)]
+-----+------------+
| NAME|age_plus_one|
+-----+------------+
|ALICE|          35|
|  BOB|          43|
|CATHY|          30|
+-----+------------+



**Explanation:**  
- `map` is Python-side (harder to optimize).  
- `select` uses JVM native expressions; vectorized and optimized.

**Optimize:**  
- Prefer Spark SQL functions over Python lambdas for speed.

## 3) filter vs where

**Goal:** Row filtering in both APIs.

In [24]:
rdd_f = rdd.filter(lambda x: x[1] > 30)
df_f = df.where("age > 30")

print(rdd_f.collect())
df_f.show()

[('alice', 34), ('bob', 42)]
+-----+---+
| name|age|
+-----+---+
|alice| 34|
|  bob| 42|
+-----+---+



**Explanation:**  
- `filter` on RDD uses Python lambdas.  
- `where`/`filter` on DataFrame compiles to JVM bytecode, supports pushdown.

**Optimize:**  
- Use DataFrame filters for datasource pushdown (Parquet/ORC).

## 4) reduce vs agg

**Goal:** Aggregation patterns.

In [25]:
# sum of ages
from operator import add
sum_age_rdd = rdd.map(lambda x: x[1]).reduce(add)

from pyspark.sql import functions as F
sum_age_df = df.agg(F.sum("age").alias("sum_age"))

print("RDD reduce sum:", sum_age_rdd)
sum_age_df.show()

RDD reduce sum: 105
+-------+
|sum_age|
+-------+
|    105|
+-------+



**Explanation:**  
- RDD `reduce` is generic but Python-side.  
- DataFrame aggregation uses column expressions; easier to compose.

**Optimize:**  
- Prefer DataFrame aggregations for performance and readability.

## 5) groupByKey vs groupBy + agg

**Goal:** Key-based aggregation efficiently.

In [26]:
pairs = spark.sparkContext.parallelize([("a",1),("a",2),("b",5),("b",1)])
# Bad: groupByKey moves all values (shuffle heavy)
gbk = pairs.groupByKey().mapValues(lambda vals: sum(vals)).collect()

# Good: reduceByKey reduces locally before shuffle
rbk = pairs.reduceByKey(lambda a,b: a+b).collect()

from pyspark.sql import functions as F
df_pairs = spark.createDataFrame([("a",1),("a",2),("b",5),("b",1)], ["k","v"])
df_agg = df_pairs.groupBy("k").agg(F.sum("v").alias("sum_v"))
print("groupByKey:", gbk)
print("reduceByKey:", rbk)
df_agg.show()

groupByKey: [('b', 6), ('a', 3)]
reduceByKey: [('b', 6), ('a', 3)]
+---+-----+
|  k|sum_v|
+---+-----+
|  a|    3|
|  b|    6|
+---+-----+



**Explanation:**  
- `groupByKey` causes wide shuffles; `reduceByKey` aggregates map-side first.  
- DataFrame `groupBy().agg()` is the preferred high-level approach.

**Optimize:**  
- Avoid `groupByKey` on large datasets; use `reduceByKey` / `aggregateByKey`.

## 6) aggregateByKey vs groupByKey

**Goal:** Custom aggregation with combiners.

In [27]:
pairs = spark.sparkContext.parallelize([("a",1),("a",2),("b",5),("b",1)])
# aggregateByKey(zero, seqOp, combOp)
abk = pairs.aggregateByKey(0, lambda acc,v: acc+v, lambda a,b: a+b).collect()
print("aggregateByKey:", abk)

aggregateByKey: [('b', 6), ('a', 3)]


**Explanation:**  
- `aggregateByKey` provides separate seq/comb steps → efficient.  
- Similar to `reduceByKey`, but more flexible (e.g., avg with (sum,count)).

**Optimize:**  
- Use combiners to reduce data shuffled.

## 7) Join: RDD vs DataFrame join

**Goal:** Compare joining APIs.

In [28]:
a = spark.sparkContext.parallelize([("u1","Alice"),("u2","Bob")])
b = spark.sparkContext.parallelize([("u1",34),("u3",20)])
rdd_join = a.join(b).collect()

dfa = spark.createDataFrame([("u1","Alice"),("u2","Bob")], ["id","name"])
dfb = spark.createDataFrame([("u1",34),("u3",20)], ["id","age"])
df_join = dfa.join(dfb, on="id", how="inner")

print("RDD join:", rdd_join)
df_join.show()

RDD join: [('u1', ('Alice', 34))]
+---+-----+---+
| id| name|age|
+---+-----+---+
| u1|Alice| 34|
+---+-----+---+



**Explanation:**  
- RDD `join` handles key-value pairs; no schema.  
- DataFrame `join` supports many join types, null handling, and Catalyst optimization.

**Optimize:**  
- Broadcast small table: `from pyspark.sql.functions import broadcast`.

## 8) Caching and storage levels (RDD vs DF)

**Goal:** Persist results for reuse.

In [29]:
from pyspark import StorageLevel

rdd_cached = rdd.persist(StorageLevel.MEMORY_ONLY)
df_cached = df.persist(StorageLevel.MEMORY_ONLY)

print("RDD cached count:", rdd_cached.count())
print("DF cached count:", df_cached.count())

rdd_cached.unpersist()
df_cached.unpersist()

RDD cached count: 3
DF cached count: 3


DataFrame[name: string, age: bigint]

**Explanation:**  
- Both APIs support `persist`/`cache`.  
- DataFrame cache integrates with SQL tab and UI.

**Optimize:**  
- Cache only if reused often; choose appropriate StorageLevel.

## 9) Partitioning: repartition/coalesce

**Goal:** Control parallelism and shuffle.

In [30]:
rdd8 = rdd.repartition(8)      # full shuffle
rdd2 = rdd8.coalesce(2)         # shrink without full shuffle
print(rdd8.getNumPartitions(), rdd2.getNumPartitions())

df8 = df.repartition(8, "age")  # partition by column (hash)
df2 = df8.coalesce(2)
print(df8.rdd.getNumPartitions(), df2.rdd.getNumPartitions())

8 2
8 2


**Explanation:**  
- DataFrames can repartition by column for better downstream joins.  
- Coalesce is cheaper when reducing partitions.

**Optimize:**  
- Align partitioning with join keys and downstream writes.

## 10) Schema & types

**Goal:** Strong typing vs Python objects.

In [31]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([StructField("name", StringType()), StructField("age", IntegerType())])
df_typed = spark.createDataFrame(data, schema=schema)
df_typed.printSchema()
df_typed.show()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)

+-----+---+
| name|age|
+-----+---+
|alice| 34|
|  bob| 42|
|cathy| 29|
+-----+---+



**Explanation:**  
- DataFrames carry schema → better validation and pushdown.  
- RDDs are flexible but opaque to Catalyst.

**Optimize:**  
- Define explicit schema when reading files to avoid inference cost/errors.

## 11) UDF vs built-in functions

**Goal:** Prefer built-ins for speed.

In [32]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf, col, length

@udf(IntegerType())
def py_len(s):
    return len(s) if s is not None else None

df_udf = df.select(py_len(col("name")).alias("len"))
df_builtin = df.select(length(col("name")).alias("len"))
df_udf.explain()
df_builtin.explain()

== Physical Plan ==
*(2) Project [pythonUDF0#280 AS len#276]
+- BatchEvalPython [py_len(name#101)#275], [pythonUDF0#280]
   +- *(1) Project [name#101]
      +- *(1) Scan ExistingRDD[name#101,age#102L]


== Physical Plan ==
*(1) Project [length(name#101) AS len#278]
+- *(1) Scan ExistingRDD[name#101,age#102L]




**Explanation:**  
- UDFs block many optimizations and are slower.  
- Built-ins generate vectorized JVM code.

**Optimize:**  
- Reserve UDFs for logic not expressible with built-ins; use Pandas UDFs if needed.

## 12) Actions: collect vs show/count

**Goal:** Avoid bringing all data to driver.

In [33]:
print("count():", df.count())
df.show(5)  # prints sample rows
# Dangerous on big data:
# rows = df.collect()  # avoid unless dataset is small

count(): 3
+-----+---+
| name|age|
+-----+---+
|alice| 34|
|  bob| 42|
|cathy| 29|
+-----+---+



**Explanation:**  
- `collect()` pulls all data to the driver → OOM risk.  
- `show()` and `take()` are safer for inspection.

**Optimize:**  
- For sampling, use `df.sample(fraction)` and `limit()`.

## 13) Sorting: sortBy vs orderBy

**Goal:** Sorting in both APIs.

In [34]:
rdd_sorted = rdd.sortBy(lambda x: x[1], ascending=False)
print(rdd_sorted.take(3))

df_sorted = df.orderBy(df.age.desc())
df_sorted.show()

[('bob', 42), ('alice', 34), ('cathy', 29)]
+-----+---+
| name|age|
+-----+---+
|  bob| 42|
|alice| 34|
|cathy| 29|
+-----+---+



**Explanation:**  
- RDD `sortBy` needs a key extractor function.  
- DataFrame `orderBy` integrates with Catalyst and pushdown (for some sources).

**Optimize:**  
- Limit rows after sort where possible to reduce shuffle cost.

## 14) SQL queries on DataFrames

**Goal:** Register temp views and query with SQL.

In [35]:
df.createOrReplaceTempView("people")
res = spark.sql("""
SELECT name, age, CASE WHEN age >= 35 THEN 'senior' ELSE 'junior' END AS band
FROM people
WHERE age >= 30
ORDER BY age DESC
""")
res.show()

+-----+---+------+
| name|age|  band|
+-----+---+------+
|  bob| 42|senior|
|alice| 34|junior|
+-----+---+------+



**Explanation:**  
- Temp views enable SQL over DataFrames.  
- SQL and DataFrame DSL are equivalent; pick what’s clearer.

**Optimize:**  
- Use ANSI SQL mode for predictable semantics; set `spark.sql.ansi.enabled=true`.

## 15) explain(): plans and costs

**Goal:** Introspect your queries to understand performance.

In [36]:
from pyspark.sql import functions as F
q = (df.withColumn("age2", F.col("age")+2)
       .where(F.col("age")>30)
       .groupBy().agg(F.avg("age2").alias("avg_age2")))
q.explain(True)

== Parsed Logical Plan ==
'Aggregate [avg('age2) AS avg_age2#331]
+- Filter (age#102L > cast(30 as bigint))
   +- Project [name#101, age#102L, (age#102L + cast(2 as bigint)) AS age2#323L]
      +- LogicalRDD [name#101, age#102L], false

== Analyzed Logical Plan ==
avg_age2: double
Aggregate [avg(age2#323L) AS avg_age2#331]
+- Filter (age#102L > cast(30 as bigint))
   +- Project [name#101, age#102L, (age#102L + cast(2 as bigint)) AS age2#323L]
      +- LogicalRDD [name#101, age#102L], false

== Optimized Logical Plan ==
Aggregate [avg(age2#323L) AS avg_age2#331]
+- Project [(age#102L + 2) AS age2#323L]
   +- Filter (isnotnull(age#102L) AND (age#102L > 30))
      +- LogicalRDD [name#101, age#102L], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[avg(age2#323L)], output=[avg_age2#331])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=629]
      +- HashAggregate(keys=[], functions=[partial_avg(age2#323L)], output=[sum#335, co

**Explanation:**  
- `explain(True)` shows parsed/analyzed/optimized/physical plans.  
- Learn to read `Project`, `Filter`, `Exchange`, `HashAggregate` nodes.

**Optimize:**  
- Remove unnecessary columns early; watch for Exchanges (shuffles).

# Part 3 — Setting up PySpark (15 examples)

## 1) Install Java & PySpark (Colab-style)

**What it is:** Minimal setup on Colab (or fresh machine with apt).

In [37]:
# If on Colab or Debian/Ubuntu:
# !apt-get update -qq
# !apt-get install -y openjdk-11-jdk -qq
# !pip install -q pyspark==3.5.1 findspark
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["PYSPARK_PYTHON"] = "python3"
# import findspark; findspark.init()
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("setup-colab").getOrCreate()
# spark.range(3).show()

**Explanation:**  
- Installs Java 11 + PySpark and sets environment variables.  
- `findspark` helps locate Spark on sys.path.

**Optimize:**  
- Pin PySpark version compatible with your Spark cluster.

## 2) Local install via pip

**What it is:** Use `pip install pyspark` on local machine.

In [38]:
# In a terminal (notebook magic shown for reference):
# !pip install pyspark==3.5.1

# Then in Python:
# from pyspark.sql import SparkSession
# spark = (SparkSession.builder
#          .appName("local-setup")
#          .master("local[*]")
#          .getOrCreate())
# print(spark.version)

**Explanation:**  
- `pip install pyspark` bundles client libs.  
- For cluster features (YARN/k8s), install a full Spark distro.

**Optimize:**  
- Keep Java & Spark versions aligned (e.g., Spark 3.5 supports Java 8/11/17).

## 3) Configure SparkSession with common configs

**What it is:** Set memory, shuffle partitions, serializer, Arrow.

In [39]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName("configured-session")
         .master("local[*]")
         .config("spark.sql.shuffle.partitions", "8")
         .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .getOrCreate())
print("Shuffle partitions:", spark.conf.get("spark.sql.shuffle.partitions"))

Shuffle partitions: 8


**Explanation:**  
- `KryoSerializer` is often faster.  
- Arrow speeds Pandas UDF and toPandas/fromPandas conversions.

**Optimize:**  
- Tune `spark.sql.shuffle.partitions` to dataset size; enable AQE.

## 4) Logging level

**What it is:** Reduce verbosity in notebooks.

In [40]:
spark.sparkContext.setLogLevel("WARN")
spark.range(5).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



**Explanation:**  
- Log level can be `ERROR`, `WARN`, `INFO`, `DEBUG`.  
- Lower noise makes debugging easier.

**Optimize:**  
- Use per-class loggers via `log4j.properties` for fine control.

## 5) Verify Java & Spark versions

**What it is:** Sanity checks for environment.

In [41]:
import os, sys
print("JAVA_HOME:", os.environ.get("JAVA_HOME"))
print("Python:", sys.version)
print("Spark version:", spark.version)

JAVA_HOME: None
Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Spark version: 3.5.1


**Explanation:**  
- Mismatched versions are the #1 setup issue.  
- Confirm `JAVA_HOME` and Spark versions early.

**Optimize:**  
- Use `pyenv`/`conda` to isolate Python; script environment checks.

## 6) Reading data (CSV/Parquet)

**What it is:** Basic file I/O with schema inference.

In [42]:
from pyspark.sql import functions as F

# Example: synthetic CSV/Parquet paths (replace with your paths)
# df_csv = spark.read.option("header","true").csv("/path/to/file.csv")
# df_parq = spark.read.parquet("/path/to/folder_or_file.parquet")
# df_csv.select("colA","colB").where(F.col("colA").isNotNull()).show(5)

**Explanation:**  
- Use `option()` for CSV parsing (delimiter, header, inferSchema).  
- Prefer Parquet/ORC for columnar pushdown.

**Optimize:**  
- Provide explicit schema for CSV to avoid costly inference.

## 7) Write data (partitioned)

**What it is:** Save DataFrames with partitioning for scalable reads.

In [43]:
# out = "/tmp/output_table"  # adjust for your environment
# (spark.range(0,1000)
#  .withColumn("bucket", (F.col("id")%5).cast("int"))
#  .write.mode("overwrite")
#  .partitionBy("bucket")
#  .parquet(out))

**Explanation:**  
- Partition columns create directory layout by value.  
- Helps pruning on reads.

**Optimize:**  
- Avoid too many small files; adjust `repartition` before writes.

## 8) Using package/jars (e.g., spark-xml)

**What it is:** Add external data source connectors.

In [44]:
# spark = (SparkSession.builder
#          .appName("with-xml")
#          .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.18.0")
#          .getOrCreate())
# df_xml = spark.read.format("xml").option("rowTag","book").load("/path/books.xml")

**Explanation:**  
- `spark.jars.packages` pulls Maven coords at session start.  
- Enables formats like XML, Delta, etc.

**Optimize:**  
- Pin versions compatible with your Spark Scala version (2.12 vs 2.13).

## 9) Enable Hive support

**What it is:** Metastore-backed tables and SQL features.

In [45]:
# spark = (SparkSession.builder
#          .appName("hive-enabled")
#          .enableHiveSupport()
#          .getOrCreate())
# spark.sql("SHOW DATABASES").show()

**Explanation:**  
- Hive support enables external metastore and ACID tables (depending on distro).

**Optimize:**  
- Configure `hive.metastore.uris` and warehouse dir before creating session.

## 10) Spark-submit basics

**What it is:** Run jobs outside notebooks.

In [46]:
# From shell:
# spark-submit --master local[4] --conf spark.sql.shuffle.partitions=8 your_job.py
#
# Inside `your_job.py`, create SparkSession and run transformations/actions.

**Explanation:**  
- `spark-submit` packages your job (and dependencies) for execution.  
- Use `--jars`/`--packages` to add connectors.

**Optimize:**  
- Provide `--files` for configs; tune executor cores/memory for cluster.

## 11) Arrow & Pandas interoperability

**What it is:** Speed up to/from Pandas operations.

In [47]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled","true")
pdf = spark.range(10).toPandas()
print(pdf.head())
df_back = spark.createDataFrame(pdf)
df_back.show()

   id
0   0
1   1
2   2
3   3
4   4
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



**Explanation:**  
- Arrow provides columnar zero-copy-ish conversions.  
- Useful for ML prep or plotting in Python.

**Optimize:**  
- Ensure Arrow versions compatible between PySpark and PyArrow.

## 12) Broadcast joins via hint

**What it is:** Force broadcast of small tables.

In [48]:
from pyspark.sql.functions import broadcast

big = spark.range(0, 100000)
small = spark.createDataFrame([(1,'x'),(2,'y')], ["id","label"])
joined = big.join(broadcast(small), big.id == small.id, "left")
joined.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [id#353L], [id#355L], LeftOuter, BuildRight, false
   :- Range (0, 100000, step=1, splits=2)
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=662]
      +- Filter isnotnull(id#355L)
         +- Scan ExistingRDD[id#355L,label#356]




**Explanation:**  
- `broadcast()` hint triggers map-side join when small table fits executor memory.

**Optimize:**  
- Control threshold via `spark.sql.autoBroadcastJoinThreshold`.

## 13) Troubleshooting: Java gateway error

**What it is:** Common fix for `Java gateway process exited` error.

In [49]:
# Ensure JAVA_HOME is set and compatible Java is installed:
# import os
# os.environ["JAVA_HOME"] = "/path/to/your/java/home"
# Restart the kernel/session after setting environment variables.

**Explanation:**  
- PySpark uses Py4J to talk to JVM. If Java is missing/mismatched, gateway fails.

**Optimize:**  
- Script environment setup; validate versions before starting SparkSession.

## 14) Memory tuning

**What it is:** Control driver/executor memory for larger workloads.

In [50]:
# Example in spark-submit:
# --driver-memory 4g --executor-memory 4g --executor-cores 2
#
# Or via builder:
# spark = (SparkSession.builder
#          .config("spark.driver.memory","4g")
#          .config("spark.executor.memory","4g")
#          .getOrCreate())

**Explanation:**  
- Memory/cores directly affect throughput and GC behavior.

**Optimize:**  
- Size to avoid GC thrash; monitor in Spark UI.

## 15) Check Spark UI & History Server

**What it is:** Monitoring running and completed apps.

In [51]:
print("Spark UI (if available):", spark.sparkContext.uiWebUrl)
# Enable event logs in production (configure before session start):
# spark = (SparkSession.builder
#          .config("spark.eventLog.enabled","true")
#          .config("spark.eventLog.dir","file:/tmp/spark-events")  # or hdfs/s3
#          .getOrCreate())

Spark UI (if available): http://8e72ff8dbcad:4040


**Explanation:**  
- Spark UI shows stages, SQL, storage, env, executors.  
- History Server reads event logs for completed apps.

**Optimize:**  
- Keep event logs in durable storage; rotate/log cleanup policies.